In [24]:
import os
from dotenv import load_dotenv
# Classification for the base classes

load_dotenv()

# Specify the path to the dataset directory
dataset_dir = os.getenv('DATASET_DIR')

target_size = (128, 128)

In [38]:

def explore_directory(directory):
    classes = []
    with os.scandir(directory) as entries:
        for entry in entries:
            if entry.is_dir():
                classes.append(entry.name)
    return classes

def recursive_class_output(directory, indentation=""):
    result = {}
    classes = explore_directory(directory)
    for class_name in classes:
        subdirectory = os.path.join(directory, class_name)
        result[class_name] = recursive_class_output(subdirectory, indentation + "    ")
    return result

In [39]:

# Initialize lists to store the images and labels
labels = recursive_class_output(dataset_dir)

{'animal': {'big': {}, 'small': {}}, 'car': {'agriculture_vehicle': {}, 'bus': {}, 'sportscar': {}}, 'lane': {'curvy': {}, 'straight': {}}, 'pedestrian': {'adult': {}, 'child': {}, 'handicaped': {}, 'senior': {}}, 'traffic_sign': {'danger': {}, 'information': {}, 'restriction': {}, 'speed_limit': {}, 'yield': {}}, 'trafic_light': {'green': {}, 'red': {}, 'yellow': {}}}
